In [8]:
# Automatically reload changes to external code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In this assignment, you will solve a classic control problem - CartPole using policy gradient methods.

Policy gradient method is a family of RL algorithms that parameterizes the policy directly. The simplest advantage that policy parameterization may have over action-value parameterization is that the policy may be a simpler function to approximate.

First, you will implement the "vanilla" policy gradient method, i.e., a method that repeatedly computes **unbiased** estimates $\hat{g}$ of $\nabla_{\theta} E[\sum_t r_t]$ and takes gradient ascent steps $\theta \rightarrow \theta + \epsilon \hat{g}$ so as to increase the total rewards collected in each episode. To make sure our code can solve multiple MDPs with different policy parameterizations, provided code follows an OOP manner and represents MDP and Policy as classes.

The following code constructs an instance of the MDP using OpenAI gym.

In [9]:
import gym
import tensorflow as tf
import numpy as np
from policy_gradient import util
from policy_gradient.policy import CategoricalPolicy
from policy_gradient.baselines.linear_feature_baseline import LinearFeatureBaseline

np.random.seed(0)
tf.set_random_seed(0)

# CartPole-v0 is a MDP with finite state and action space. 
# In this environment, A pendulum is attached by an un-actuated joint to a cart, 
# and the goal is to prevent it from falling over. You can apply a force of +1 or -1 to the cart.
# A reward of +1 is provided for every timestep that the pendulum remains upright. 
# To visualize CartPole-v0, please see https://gym.openai.com/envs/CartPole-v0

env = gym.make('CartPole-v0')

[2017-11-09 19:32:30,767] Making new env: CartPole-v0


## Problem 1: construct a neural network to represent policy

Make sure you know how to construct neural network using tensorflow.

1. Open **homework2/policy_gradient/policy.py**.
2. Follow the instruction of Problem 1.

## Problem 2: compute the surrogate loss

If there are $N$ episodes in an iteration, then for $i$ th episode we define $R_t^i = \sum_{{t^′}=t}^T \gamma^{{t^′}-t}r(s_{t^′}, a_{t^′})$ as the accumulated discounted rewards from timestep $t$ to the end of that episode, where $\gamma$ is the discount rate.

The pseudocode for the REINFORCE algorithm is as below:

1. Initialize policy $\pi$ with parameter $\theta_1$.
2. For iteration $k = 1, 2, ...$:
    * Sample N episodes $\tau_1, \tau_2, ..., \tau_N$ under the current policy $\theta_k$, where $\tau_i =(s_i^t,a_i^t,R_i^t)_{t=0}^{T−1}$. Note that the last state is dropped since no action is taken after observing the last state.
    * Compute the empirical policy gradient using formula: $$\hat{g} = E_{\pi_\theta}[\nabla_{\theta} log\pi_\theta(a_t^i | s_t^i) R_t^i]$$
    * Take a gradient step: $\theta_{k+1} = \theta_k + \epsilon \hat{g}$.
    
    
Note that we can transform the policy gradient formula as

$$\hat{g} = \nabla_{\theta} \frac{1}{(NT)}(\sum_{i=1}^N \sum_{t=0}^T log\pi_\theta(a_t^i | s_t^i) *R_t^i)$$

and $L(\theta) = \frac{1}{(NT)}(\sum_{i=1}^N \sum_{t=0}^T log\pi_\theta(a_t^i | s_t^i) *R_t^i)$ is called the surrogate loss. 

We can first construct the computation graph for $L(\theta)$, and then take its gradient as the empirical policy gradient.


1. Open **homework2/policy_gradient/policy.py**.
2. Follow the instruction of Problem 2.

In [10]:
tf.reset_default_graph()
sess = tf.Session()
# Construct a neural network to represent policy which maps observed state to action. 
in_dim = util.flatten_space(env.observation_space)
out_dim = util.flatten_space(env.action_space)
hidden_dim = 8

# Initialize your policy
with tf.variable_scope("policy"):
    opt_p = tf.train.AdamOptimizer(learning_rate=0.01)
    policy = CategoricalPolicy(in_dim, out_dim, hidden_dim, opt_p, sess)


D:\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# Problem 3

Use baseline to reduce the variance of our gradient estimate.

Change the loss term: 

$$L(\theta) = \frac{1}{(NT)}(\sum_{i=1}^N \sum_{t=0}^T log\pi_\theta(a_t^i | s_t^i) *R_t^i)$$

into this one

$$L(\theta) = \frac{1}{(NT)}(\sum_{i=1}^N \sum_{t=0}^T log\pi_\theta(a_t^i | s_t^i) *(R_t^i-V_t^i))$$

where $V_t^i$ is the baseline prediction for at the $t$ timestep $i^{th}$ trajectory. In this part, we use a simple linear function to parameterize value function. (You're encouraged to see the details in ```policy_gradient/baselines/linear_feature_baseline.py```)

1. Fill in the function `process_paths` of class `PolicyOptimizer` below.

In [11]:
class PolicyOptimizer(object):
    def __init__(self, env, policy, baseline, n_iter, n_episode, path_length,
        discount_rate=.99):

        self.policy = policy
        self.baseline = baseline
        self.env = env
        self.n_iter = n_iter
        self.n_episode = n_episode
        self.path_length = path_length
        self.discount_rate = discount_rate

    def sample_path(self):
        obs = []
        actions = []
        rewards = []
        ob = self.env.reset()

        # sample a batch of trajectory
        for _ in range(self.path_length):
            a = self.policy.act(ob.reshape(1, -1))
            next_ob, r, done, _ = self.env.step(a)
            obs.append(ob)
            actions.append(a)
            rewards.append(r)
            ob = next_ob
            if done:
                break

        return dict(
            observations=np.array(obs),
            actions=np.array(actions),
            rewards=np.array(rewards),
        )

    def process_paths(self, paths):
        for p in paths:
            if self.baseline != None:
                b = self.baseline.predict(p)
                b[-1] = 0 # terminal state
            else:
                b = 0
            
            # `p["rewards"]` is a matrix contains the rewards of each timestep in a sample path
            r = util.discount_cumsum(p["rewards"], self.discount_rate)
            
            """
            Problem 3:

            1. Variable `b` is the values predicted by our baseline
            2. Use it to reduce variance and then assign the result to the 
                    variable `a` (baseline reduction)

            Sample solution should be only 1 line.
            """
            # YOUR CODE HERE >>>>>>
            a = r - b 
            # <<<<<<<<

            p["returns"] = r
            p["baselines"] = b
            p["advantages"] = (a - a.mean()) / (a.std() + 1e-8) # normalize

        obs = np.concatenate([ p["observations"] for p in paths ])
        actions = np.concatenate([ p["actions"] for p in paths ])
        rewards = np.concatenate([ p["rewards"] for p in paths ])
        advantages = np.concatenate([ p["advantages"] for p in paths ])

        return dict(
            observations=obs,
            actions=actions,
            rewards=rewards,
            advantages=advantages,
        )

    
    def train(self):
        loss_list = []
        avg_return_list = []
        for i in range(1, self.n_iter + 1):
            paths = []
            for _ in range(self.n_episode):
                paths.append(self.sample_path())
            data = self.process_paths(paths)
            loss = self.policy.train(data["observations"], data["actions"], data["advantages"])
            avg_return = np.mean([sum(p["rewards"]) for p in paths])
            print("Iteration {}: Average Return = {}".format(i, avg_return))
            loss_list.append(loss)
            avg_return_list.append(avg_return)
            # CartPole-v0 defines "solving" as getting average reward of 195.0 over 100 consecutive trials.
            if avg_return >= 195:
                print("Solve at {} iterations, which equals {} episodes.".format(i, i*100))
                break

            if self.baseline != None:
                self.baseline.fit(paths)
        return loss_list, avg_return_list

In [ ]:
sess.run(tf.global_variables_initializer())

n_iter = 200
n_episode = 100
path_length = 200
discount_rate = 0.99
baseline = LinearFeatureBaseline(env.spec)

po = PolicyOptimizer(env, policy, baseline, n_iter, n_episode, path_length,
                     discount_rate)

# Train the policy optimizer
loss_list, avg_return_list = po.train()

In [ ]:
util.plot_curve(loss_list, "loss")
util.plot_curve(avg_return_list, "average return")

# Verify your solutions

if you solve the problems 1~3 correctly, your will solve CartPole with roughly ~ 80 iterations.

# Problem 4

Replacing line 

`baseline = LinearFeatureBaseline(env.spec)` 

with 

`baseline = None`

can remove the baseline.

Modify the code to compare the variance and performance before and after adding baseline. And explain wht the baseline won't introduce bias. Then, write a report about your findings and explainations. 

In [41]:
sess.run(tf.global_variables_initializer())

n_iter = 200
n_episode = 100
path_length = 200
discount_rate = 0.99
baseline = None

po = PolicyOptimizer(env, policy, baseline, n_iter, n_episode, path_length,
                     discount_rate)

# Train the policy optimizer
loss_list, avg_return_list = po.train()

Iteration 1: Average Return = 26.49
Iteration 2: Average Return = 29.43
Iteration 3: Average Return = 28.48
Iteration 4: Average Return = 30.64
Iteration 5: Average Return = 31.16
Iteration 6: Average Return = 34.4
Iteration 7: Average Return = 35.76
Iteration 8: Average Return = 36.66
Iteration 9: Average Return = 38.88
Iteration 10: Average Return = 38.69
Iteration 11: Average Return = 39.65
Iteration 12: Average Return = 42.97
Iteration 13: Average Return = 40.22
Iteration 14: Average Return = 37.81
Iteration 15: Average Return = 37.11
Iteration 16: Average Return = 40.78
Iteration 17: Average Return = 45.22
Iteration 18: Average Return = 42.94
Iteration 19: Average Return = 44.96
Iteration 20: Average Return = 50.13
Iteration 21: Average Return = 43.86
Iteration 22: Average Return = 47.98
Iteration 23: Average Return = 47.68
Iteration 24: Average Return = 50.6
Iteration 25: Average Return = 48.18
Iteration 26: Average Return = 45.84
Iteration 27: Average Return = 51.54
Iteration 28

In [42]:
util.plot_curve(loss_list, "loss")
util.plot_curve(avg_return_list, "average return")

# Problem 5: Actor-Critic algorithm (with bootstrapping)

The baseline reduction lowers the variance without the cost of introducing bias. Are there any others that can reduce the variance of the policy?

In this part, you will implement a simple actor-critic algorithm. Try to overwrite the `process_paths` in `PolicyOptimizer_actor_critic`. Note that the `PolicyOptimizer_actor_critic` inherit the class `PolicyOptimizer`


We use the one-step bootstrap for the advantage function, which change the $A_t^i=(R_t^i-V_t^i)$ in problem 3 into:

$$A_t^i = r_t^i + \gamma*V_{t+1}^i - V_t^i$$

Open **policy_gradient/util.py ** to implement the function `discount_bootstrap`

If you answer is right, your will solve CartPole with roughly ~ 80 iterations.

# Problem 6: Generalized Advantage Estimation

In Problem 3, we calculate the advantage function with **the total return substract the baseline** 

In Problem 5, we calculate the advantage function based on Problem 3, and **replace the total return with the immediate reward plus estimated baseline**

Here, we use a novel advantage function called "Generalized Advantage Estimation", which introduces one hyperparameter $\lambda$ to compromise the above two estimation methods.

Assume the $\delta_t^i$ represent the i-step bootstrapping (e.g. $\delta_t^i=r_t^i + \gamma*V_{t+1}^i - V_t^i$). The generalized advantage estimation will be:

$$A_{t}^{GAE} = \sum_{l=0}^{\infty} (\gamma\lambda)^l \delta_{t+1}$$

Please see [High-Dimensional Continuous Control Using Generalized Advantage Estimation](https://arxiv.org/abs/1506.02438) for details.


If you answer is right, your will solve CartPole with roughly ~ 80 iterations.

In [36]:
# set the hyperparameter for generalized advantage estimation (GAE)
LAMBDA = 0.98 # \lambda
class PolicyOptimizer_actor_critic(PolicyOptimizer):
    def __init__(self, env, policy, baseline, n_iter, n_episode, path_length,
        discount_rate=.99):
        PolicyOptimizer.__init__(self, env, policy, baseline, n_iter, n_episode, path_length,
            discount_rate=.99)
    
    def process_paths(self, paths):
        for p in paths:
            if self.baseline != None:
                b = self.baseline.predict(p)
                b[-1] = 0 # terminal state
            else:
                b = 0
            
            """
            1. Variable `b` is the reward predicted by our baseline
            2. Calculate the advantage function via one-step bootstrap
                    A(s, a) = [r(s,a,s') + \gamma*v(s')] - v(s)
            3. `target_v` specifies the target of the baseline function
            """
            r = util.discount_bootstrap(p["rewards"], self.discount_rate, b)
            target_v = util.discount_cumsum(p["rewards"], self.discount_rate)
            a = r - b
            
            """
            Problem 6 (Skip this when doing Proble): 
            
            1. in Problem 3, we calculate the advantage function with the total 
                       return substract the baseline 
               in Problem 5, we calculate the advantage function based on Problem 3. 
                       And replace the total return with the immediate reward plus estimated baseline
            2. Here, we use a novel advantage function called "Generalized Advantage Estimation", 
                       which introduces one hyperparameter to balance the above two estimation methods.
            Currently, the variable `LAMBDA` is the default hyperparameter for GAE. 
                        (You're encouraged to try other \lambda=[0,1])
            Sample solution should be only 1 line. (you can use `util.discount` in policy_gradient/util.py)
            """
            # YOUR CODE HERE >>>>>>>>
            a = util.discount(a, self.discount_rate * LAMBDA)
            # <<<<<<<
            p["returns"] = target_v
            p["baselines"] = b
            p["advantages"] = (a - a.mean()) / (a.std() + 1e-8) # normalize

        obs = np.concatenate([ p["observations"] for p in paths ])
        actions = np.concatenate([ p["actions"] for p in paths ])
        rewards = np.concatenate([ p["rewards"] for p in paths ])
        advantages = np.concatenate([ p["advantages"] for p in paths ])

        return dict(
            observations=obs,
            actions=actions,
            rewards=rewards,
            advantages=advantages,
        )

In [38]:
sess.run(tf.global_variables_initializer())

n_iter = 200
n_episode = 100
path_length = 200
discount_rate = 0.99
# reinitialize the baseline function
baseline = LinearFeatureBaseline(env.spec) 
sess.run(tf.global_variables_initializer())
po = PolicyOptimizer_actor_critic(env, policy, baseline, n_iter, n_episode, path_length,
                     discount_rate)

# Train the policy optimizer
loss_list, avg_return_list = po.train()

Iteration 1: Average Return = 35.9
Iteration 2: Average Return = 36.56
Iteration 3: Average Return = 44.01
Iteration 4: Average Return = 38.99
Iteration 5: Average Return = 50.8
Iteration 6: Average Return = 50.16
Iteration 7: Average Return = 49.52
Iteration 8: Average Return = 51.58
Iteration 9: Average Return = 52.02
Iteration 10: Average Return = 57.15
Iteration 11: Average Return = 60.54
Iteration 12: Average Return = 67.6
Iteration 13: Average Return = 66.56
Iteration 14: Average Return = 71.31
Iteration 15: Average Return = 88.19
Iteration 16: Average Return = 81.68
Iteration 17: Average Return = 92.36
Iteration 18: Average Return = 101.59
Iteration 19: Average Return = 107.51
Iteration 20: Average Return = 129.83
Iteration 21: Average Return = 131.6
Iteration 22: Average Return = 136.74
Iteration 23: Average Return = 137.16
Iteration 24: Average Return = 150.07
Iteration 25: Average Return = 147.15
Iteration 26: Average Return = 145.06
Iteration 27: Average Return = 157.3
Itera

In [39]:
util.plot_curve(loss_list, "loss")
util.plot_curve(avg_return_list, "average return")